#PIP Install


In [ ]:
!pip install chromadb ollama langchain langchain-community langchain-chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.7 MB/s eta 0:00:

#Install Ollama

In [ ]:
# enable support for gpu
!apt-get install lshw

# download ollama
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 791 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 791 kB in 2s (386 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 126109 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.996aaad9c

##Start Ollama

In [ ]:
# start ollama in background to orevent blocking the terminal
!nohup ollama serve &

nohup: appending output to 'nohup.out'


##Pull Models

In [ ]:
!ollama pull llama3

In [ ]:
!ollama pull all-minilm:l6-v2

In [ ]:
!ollama pull nomic-embed-text

##List Models

In [ ]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    
llama3:latest              365c0bd3c000    4.7 GB    9 seconds ago             


#CSV Analysis and Response Code

In [ ]:
# Install required packages
!pip install pandas langchain langchain_community
!pip install python-docx # Install python-docx
!pip install chromadb
!pip install pypdf pillow unstructured ollama

In [ ]:
# Imports
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.output_parsers.pandas_dataframe import PandasDataFrameOutputParser
import pandas as pd

In [ ]:
df = pd.read_csv("/content/cleaned_events.csv")
#df

FileNotFoundError: [Errno 2] No such file or directory: '/content/cleaned_events.csv'

In [ ]:
# Imports
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.schema import BaseOutputParser
import pandas as pd
import re
from pydantic import Field, ConfigDict
import numpy as np

# Load your dataframe
df = pd.read_csv("/content/cleaned_events.csv")

# Define instruction string
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
    "6. Make sure the expression is syntactically correct Python code.\n"
    "7. Use only columns that exist in the dataframe.\n"
)

# Define pandas prompt string with better context
pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Available columns: {columns}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Important: Only use columns that exist in the dataframe. Make sure your expression is valid Python syntax.\n\n"
    "Expression:"
)

# Define response synthesis prompt string
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

# Initialize LLaMA 3 model
llm = Ollama(model="llama3")

# Create a custom output parser for pandas instructions
class PandasInstructionParser(BaseOutputParser):
    df: pd.DataFrame = Field(default=None)
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def __init__(self, dataframe: pd.DataFrame, **kwargs):
        super().__init__(**kwargs)
        self.df = dataframe

    def parse(self, text: str):
        # Extract the pandas expression from the LLM output
        text = text.strip()

        # Remove markdown code blocks if present
        if "```" in text:
            code_match = re.search(r'```(?:python)?\s*(.*?)\s*```', text, re.DOTALL)
            if code_match:
                text = code_match.group(1).strip()

        # Remove any leading/trailing quotes
        text = text.strip('"\'')

        # Clean up common issues
        lines = text.split('\n')
        # Get the last non-empty line as the expression
        expression = None
        for line in reversed(lines):
            line = line.strip()
            if line and not line.startswith('#') and not line.startswith('import'):
                expression = line
                break

        if not expression:
            expression = text

        try:
            # Create a safe execution environment
            safe_dict = {
                "df": self.df,
                "pd": pd,
                "np": np,
                "__builtins__": {}
            }

            print(f"Executing expression: {expression}")
            result = eval(expression, safe_dict)
            return result
        except Exception as e:
            return f"Error executing pandas expression: {str(e)}\nExpression: {expression}"

    @property
    def _type(self) -> str:
        return "pandas_instruction"

# Initialize components
pandas_prompt = PromptTemplate(
    template=pandas_prompt_str,
    input_variables=["query_str"],
    partial_variables={
        "instruction_str": instruction_str,
        "df_str": str(df.head(5)),
        "columns": str(list(df.columns))
    }
)

pandas_output_parser = PandasInstructionParser(df)

response_synthesis_prompt = PromptTemplate(
    template=response_synthesis_prompt_str,
    input_variables=["query_str", "pandas_instructions", "pandas_output"]
)

# Create the complete pipeline function using modern LangChain approach
def run_query_pipeline(query_str):
    # Step 1: Generate pandas instructions and execute them
    print(f"Processing query: {query_str}")

    # Use the modern invoke method instead of deprecated __call__
    pandas_instructions_raw = llm.invoke(pandas_prompt.format(query_str=query_str))
    print(f"Generated pandas instructions: {pandas_instructions_raw}")

    # Parse and execute the pandas instructions
    pandas_output = pandas_output_parser.parse(pandas_instructions_raw)
    print(f"Pandas output: {pandas_output}")

    # Step 2: Synthesize final response using modern approach
    response_input = {
        "query_str": query_str,
        "pandas_instructions": pandas_instructions_raw,
        "pandas_output": str(pandas_output)
    }

    # Use invoke instead of deprecated run method
    final_response = llm.invoke(response_synthesis_prompt.format(**response_input))

    return final_response

# Alternative function using the new RunnableSequence approach
def run_query_pipeline_modern(query_str):
    """Modern implementation using RunnableSequence"""
    print(f"Processing query: {query_str}")

    # Create the sequence: prompt | llm | parser
    pandas_sequence = pandas_prompt | llm | pandas_output_parser

    # Execute the pandas sequence
    pandas_result = pandas_sequence.invoke({"query_str": query_str})
    print(f"Pandas output: {pandas_result}")

    # Create response sequence
    response_sequence = response_synthesis_prompt | llm

    # Execute response sequence
    final_response = response_sequence.invoke({
        "query_str": query_str,
        "pandas_instructions": "Generated pandas code",
        "pandas_output": str(pandas_result)
    })

    return final_response

# Example usage
if __name__ == "__main__":
    # Display the dataframe info first
    print("DataFrame Info:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nSample data:")
    print(df.head())

    # Run queries appropriate for the events data
    print("\n" + "="*50)
    response1 = run_query_pipeline("How many events are there in total?")
    print(f"\nFinal Response: {response1}")

    print("\n" + "="*50)
    response2 = run_query_pipeline("What are the most common locations for events?")
    print(f"\nFinal Response: {response2}")

    print("\n" + "="*50)
    response3 = run_query_pipeline("Whats happening in first week of june")
    print(f"\nFinal Response: {response3}")

# RAG

##System

In [ ]:
#!/usr/bin/env python3
"""
Complete Modular RAG System with ChromaDB and Llama3
Requires: pip install chromadb ollama langchain langchain-community langchain-chroma
"""

import os
import json
import hashlib
import time
from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional
from pathlib import Path
from dataclasses import dataclass
from enum import Enum

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    TextLoader, PyPDFLoader, CSVLoader,
    UnstructuredMarkdownLoader, UnstructuredHTMLLoader
)
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

# Core data structures
class QueryType(Enum):
    FACTUAL = "factual"
    ANALYTICAL = "analytical"
    OPINION = "opinion"
    CONTEXTUAL = "contextual"

@dataclass
class Document:
    content: str
    metadata: Dict[str, Any]
    source: str
    chunk_id: Optional[str] = None

@dataclass
class RetrievalResult:
    documents: List[Document]
    scores: List[float]
    strategy_used: str
    metadata: Dict[str, Any] = None

@dataclass
class QueryClassification:
    query_type: QueryType
    confidence: float
    reasoning: str

@dataclass
class RAGResponse:
    answer: str
    sources: List[Document]
    query_type: QueryType
    strategy_used: str
    confidence: float
    metadata: Dict[str, Any] = None

# Abstract base classes
class QueryClassifier(ABC):
    @abstractmethod
    def classify(self, query: str) -> QueryClassification:
        pass

class RetrievalStrategy(ABC):
    @abstractmethod
    def retrieve(self, query: str, top_k: int = 5) -> RetrievalResult:
        pass

class ResponseGenerator(ABC):
    @abstractmethod
    def generate(self, query: str, retrieval_result: RetrievalResult,
                query_type: QueryType) -> str:
        pass

class ResponseEvaluator(ABC):
    @abstractmethod
    def evaluate(self, query: str, response: str, sources: List[Document]) -> Dict[str, Any]:
        pass

# Concrete implementations
class LLMQueryClassifier(QueryClassifier):
    def __init__(self, llm):
        self.llm = llm

    def classify(self, query: str) -> QueryClassification:
        classification_prompt = f"""
        Classify the following query into one of these categories:

        1. FACTUAL: Asking for specific facts, definitions, or direct information
        2. ANALYTICAL: Requiring analysis, comparison, or synthesis of information
        3. OPINION: Seeking subjective views, recommendations, or interpretations
        4. CONTEXTUAL: Needing understanding of relationships, context, or broader implications

        Query: {query}

        Respond in this exact format:
        Category: [FACTUAL/ANALYTICAL/OPINION/CONTEXTUAL]
        Confidence: [0.0-1.0]
        Reasoning: [Brief explanation]
        """

        try:
            response = self.llm.invoke(classification_prompt).strip()

            # Parse response
            lines = response.split('\n')
            category = QueryType.FACTUAL  # default
            confidence = 0.5
            reasoning = "Default classification"

            for line in lines:
                if line.startswith('Category:'):
                    cat_str = line.split(':', 1)[1].strip().upper()
                    try:
                        category = QueryType(cat_str.lower())
                    except ValueError:
                        category = QueryType.FACTUAL
                elif line.startswith('Confidence:'):
                    try:
                        confidence = float(line.split(':', 1)[1].strip())
                    except ValueError:
                        confidence = 0.5
                elif line.startswith('Reasoning:'):
                    reasoning = line.split(':', 1)[1].strip()

            return QueryClassification(category, confidence, reasoning)
        except Exception as e:
            print(f"Error in classification: {e}")
            return QueryClassification(QueryType.FACTUAL, 0.5, "Error in classification")

class FactualRetrievalStrategy(RetrievalStrategy):
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def retrieve(self, query: str, top_k: int = 5) -> RetrievalResult:
        try:
            retriever = self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": top_k}
            )

            docs = retriever.get_relevant_documents(query)
            documents = [Document(
                content=doc.page_content,
                metadata=doc.metadata,
                source=doc.metadata.get('source', 'unknown')
            ) for doc in docs]
            scores = [1.0] * len(documents)

            return RetrievalResult(
                documents=documents,
                scores=scores,
                strategy_used="factual",
                metadata={"search_type": "similarity"}
            )
        except Exception as e:
            print(f"Error in factual retrieval: {e}")
            return RetrievalResult([], [], "factual", {"error": str(e)})

class AnalyticalRetrievalStrategy(RetrievalStrategy):
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def retrieve(self, query: str, top_k: int = 8) -> RetrievalResult:
        try:
            retriever = self.vectorstore.as_retriever(
                search_type="mmr",
                search_kwargs={"k": top_k, "fetch_k": top_k * 2, "lambda_mult": 0.5}
            )

            docs = retriever.get_relevant_documents(query)
            documents = [Document(
                content=doc.page_content,
                metadata=doc.metadata,
                source=doc.metadata.get('source', 'unknown')
            ) for doc in docs]
            scores = [1.0] * len(documents)

            return RetrievalResult(
                documents=documents,
                scores=scores,
                strategy_used="analytical",
                metadata={"search_type": "mmr"}
            )
        except Exception as e:
            print(f"Error in analytical retrieval: {e}")
            return RetrievalResult([], [], "analytical", {"error": str(e)})

class OpinionRetrievalStrategy(RetrievalStrategy):
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def retrieve(self, query: str, top_k: int = 6) -> RetrievalResult:
        try:
            retriever = self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": top_k}
            )

            docs = retriever.get_relevant_documents(query)
            documents = [Document(
                content=doc.page_content,
                metadata=doc.metadata,
                source=doc.metadata.get('source', 'unknown')
            ) for doc in docs]
            scores = [1.0] * len(documents)

            return RetrievalResult(
                documents=documents,
                scores=scores,
                strategy_used="opinion",
                metadata={"search_type": "similarity_opinion"}
            )
        except Exception as e:
            print(f"Error in opinion retrieval: {e}")
            return RetrievalResult([], [], "opinion", {"error": str(e)})

class ContextualRetrievalStrategy(RetrievalStrategy):
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore

    def retrieve(self, query: str, top_k: int = 10) -> RetrievalResult:
        try:
            retriever = self.vectorstore.as_retriever(
                search_type="mmr",
                search_kwargs={"k": top_k, "fetch_k": top_k * 2, "lambda_mult": 0.3}
            )

            docs = retriever.get_relevant_documents(query)
            documents = [Document(
                content=doc.page_content,
                metadata=doc.metadata,
                source=doc.metadata.get('source', 'unknown')
            ) for doc in docs]
            scores = [1.0] * len(documents)

            return RetrievalResult(
                documents=documents,
                scores=scores,
                strategy_used="contextual",
                metadata={"search_type": "mmr_contextual"}
            )
        except Exception as e:
            print(f"Error in contextual retrieval: {e}")
            return RetrievalResult([], [], "contextual", {"error": str(e)})

class AdaptiveResponseGenerator(ResponseGenerator):
    def __init__(self, llm):
        self.llm = llm
        self.templates = {
            QueryType.FACTUAL: """
Based on the following context, provide a direct, factual answer to the question.
Be precise and cite specific information from the sources.

Context: {context}

Question: {query}

Answer:""",

            QueryType.ANALYTICAL: """
Using the provided context, analyze and synthesize the information to answer the question.
Consider multiple perspectives and provide reasoning for your conclusions.

Context: {context}

Question: {query}

Analysis:""",

            QueryType.OPINION: """
Based on the context provided, offer a balanced perspective on the question.
Present different viewpoints when available and acknowledge subjective elements.

Context: {context}

Question: {query}

Perspective:""",

            QueryType.CONTEXTUAL: """
Using the comprehensive context provided, explain the relationships and broader implications
relevant to the question. Provide background and situate the answer within the larger context.

Context: {context}

Question: {query}

Contextual Answer:"""
        }

    def generate(self, query: str, retrieval_result: RetrievalResult,
                query_type: QueryType) -> str:
        try:
            if not retrieval_result.documents:
                return "I don't have enough information to answer this question."

            context = "\n\n".join([doc.content for doc in retrieval_result.documents[:5]])
            template = self.templates.get(query_type, self.templates[QueryType.FACTUAL])

            prompt = template.format(context=context, query=query)
            response = self.llm.invoke(prompt)

            return response.strip()
        except Exception as e:
            print(f"Error in response generation: {e}")
            return f"Error generating response: {str(e)}"

class SimpleResponseEvaluator(ResponseEvaluator):
    def __init__(self, llm):
        self.llm = llm

    def evaluate(self, query: str, response: str, sources: List[Document]) -> Dict[str, Any]:
        try:
            eval_prompt = f"""
Evaluate the quality of this response on a scale of 1-10:

Query: {query}
Response: {response}

Consider:
- Relevance to the query
- Accuracy based on sources
- Completeness
- Clarity

Provide only a score from 1-10 and brief explanation.
Format: Score: X
Explanation: [brief explanation]
"""

            eval_response = self.llm.invoke(eval_prompt).strip()

            score = 5.0
            explanation = "Default evaluation"

            lines = eval_response.split('\n')
            for line in lines:
                if line.startswith('Score:'):
                    try:
                        score_str = line.split(':', 1)[1].strip()
                        score = float(score_str)
                    except:
                        pass
                elif line.startswith('Explanation:'):
                    explanation = line.split(':', 1)[1].strip()

            return {
                "score": score,
                "explanation": explanation,
                "source_count": len(sources),
                "response_length": len(response)
            }
        except Exception as e:
            return {
                "score": 5.0,
                "explanation": f"Error in evaluation: {str(e)}",
                "source_count": len(sources),
                "response_length": len(response)
            }

# Main RAG System
class ModularRAGSystem:
    def __init__(self,
                 vectorstore,
                 llm,
                 query_classifier: Optional[QueryClassifier] = None,
                 response_generator: Optional[ResponseGenerator] = None,
                 response_evaluator: Optional[ResponseEvaluator] = None):

        self.vectorstore = vectorstore
        self.llm = llm

        self.query_classifier = query_classifier or LLMQueryClassifier(llm)
        self.response_generator = response_generator or AdaptiveResponseGenerator(llm)
        self.response_evaluator = response_evaluator or SimpleResponseEvaluator(llm)

        self.retrieval_strategies = {
            QueryType.FACTUAL: FactualRetrievalStrategy(vectorstore),
            QueryType.ANALYTICAL: AnalyticalRetrievalStrategy(vectorstore),
            QueryType.OPINION: OpinionRetrievalStrategy(vectorstore),
            QueryType.CONTEXTUAL: ContextualRetrievalStrategy(vectorstore)
        }

    def process_query(self, query: str) -> RAGResponse:
        try:
            # Step 1: Classify the query
            classification = self.query_classifier.classify(query)

            # Step 2: Retrieve documents
            retrieval_strategy = self.retrieval_strategies[classification.query_type]
            retrieval_result = retrieval_strategy.retrieve(query)


            # Step 3: Generate response
            answer = self.response_generator.generate(
                query, retrieval_result, classification.query_type
            )

            # Step 4: Evaluate response
            evaluation = self.response_evaluator.evaluate(
                query, answer, retrieval_result.documents
            )

            return RAGResponse(
                answer=answer,
                sources=retrieval_result.documents,
                query_type=classification.query_type,
                strategy_used=retrieval_result.strategy_used,
                confidence=classification.confidence,
                metadata={
                    "classification": {
                        "confidence": classification.confidence,
                        "reasoning": classification.reasoning
                    },
                    "retrieval": retrieval_result.metadata,
                    "evaluation": evaluation
                }
            )
        except Exception as e:
            print(f"Error processing query: {e}")
            return RAGResponse(
                answer=f"Error processing query: {str(e)}",
                sources=[],
                query_type=QueryType.FACTUAL,
                strategy_used="error",
                confidence=0.0,
                metadata={"error": str(e)}
            )

# Document indexing system
class DocumentIndexer:
    def __init__(self, vectorstore, text_splitter=None):
        self.vectorstore = vectorstore
        self.text_splitter = text_splitter or RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )

    def get_loader_for_file(self, file_path: Path):
        suffix = file_path.suffix.lower()
        if suffix == ".txt":
            return TextLoader(str(file_path))
        elif suffix == ".pdf":
            return PyPDFLoader(str(file_path))
        elif suffix == ".csv":
            return CSVLoader(str(file_path))
        elif suffix in [".md", ".markdown"]:
            return UnstructuredMarkdownLoader(str(file_path))
        elif suffix in [".html", ".htm"]:
            return UnstructuredHTMLLoader(str(file_path))
        return None

    def index_directory(self, data_dir: str):
        data_path = Path(data_dir)
        if not data_path.exists():
            print(f"Directory {data_dir} does not exist")
            return

        print(f"Indexing documents from {data_dir}...")

        for file_path in data_path.glob("**/*"):
            if file_path.is_file() and not file_path.name.startswith("."):
                try:
                    loader = self.get_loader_for_file(file_path)
                    if loader:
                        print(f"Processing: {file_path}")
                        documents = loader.load()
                        chunks = self.text_splitter.split_documents(documents)

                        if chunks:
                            self.vectorstore.add_documents(chunks)
                            print(f"  Added {len(chunks)} chunks")
                        else:
                            print(f"  No content extracted")
                    else:
                        print(f"  Unsupported file type: {file_path}")
                except Exception as e:
                    print(f"  Error processing {file_path}: {e}")

        print("Indexing complete!")

# Setup and main execution
def setup_system(data_dir: str = "./data", persist_dir: str = "./chroma_db"):
    """Setup the complete RAG system"""
    print("Setting up RAG system...")

    # Initialize Ollama models
    print("Initializing Llama3...")
    llm = Ollama(model="llama3", temperature=0.2)

    print("Initializing embeddings...")
    embeddings = OllamaEmbeddings(model="nomic-embed-text")

    # Setup ChromaDB
    print("Setting up ChromaDB...")
    vectorstore = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )

    # Create indexer and RAG system
    indexer = DocumentIndexer(vectorstore)
    rag_system = ModularRAGSystem(vectorstore, llm)

    return rag_system, indexer

def create_sample_documents(data_dir: str = "./data"):
    """Create sample documents for testing"""
    data_path = Path(data_dir)
    data_path.mkdir(exist_ok=True)

    # Sample documents
    documents = {
        "ai_basics.txt": """
Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of intelligent behavior.
AI systems can perform tasks that typically require human intelligence, such as visual perception, speech recognition,
decision-making, and language translation.

Machine Learning is a subset of AI that enables computers to learn and improve from experience without being explicitly
programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.

Deep Learning is a specialized form of machine learning that uses neural networks with multiple layers to model and
understand complex patterns in data. It has been particularly successful in areas like image recognition and natural
language processing.
""",
        "technology_trends.txt": """
Current technology trends include the rise of artificial intelligence, cloud computing, and edge computing.
These technologies are transforming how businesses operate and how people interact with digital services.

Quantum computing represents a paradigm shift in computational power, with the potential to solve complex problems
that are currently intractable for classical computers. Major tech companies are investing heavily in quantum research.

The Internet of Things (IoT) continues to expand, connecting everyday objects to the internet and enabling new forms
of automation and data collection. Smart homes, wearable devices, and industrial IoT are driving this growth.
""",
        "business_strategy.txt": """
Digital transformation is no longer optional for businesses; it's essential for survival in the modern economy.
Companies must adapt their processes, culture, and technology to remain competitive.

Customer experience has become a key differentiator in many industries. Businesses are investing in personalization,
omnichannel approaches, and real-time customer service to meet evolving expectations.

Data-driven decision making is becoming standard practice. Organizations are leveraging analytics, machine learning,
and business intelligence tools to gain insights and make informed strategic decisions.
"""
    }

    for filename, content in documents.items():
        file_path = data_path / filename
        with open(file_path, 'w') as f:
            f.write(content)

    print(f"Created sample documents in {data_dir}")

def main():
    """Main execution function"""
    print("=== Modular RAG System with ChromaDB and Llama3 ===\n")

    # Check if Ollama is running
    try:
        test_llm = Ollama(model="llama3")
        test_llm.invoke("Hello")
        print("✓ Ollama connection successful")
    except Exception as e:
        print(f"✗ Ollama connection failed: {e}")
        print("Please ensure Ollama is running and llama3 model is installed")
        print("Run: ollama pull llama3")
        print("Run: ollama pull nomic-embed-text")
        return

    # Setup directories
    data_dir = "./my_documents"
    persist_dir = "./chroma_db"

    # Create sample documents if data directory doesn't exist
    if not Path(data_dir).exists():
        create_sample_documents(data_dir)

    # Setup system
    rag_system, indexer = setup_system(data_dir, persist_dir)

    # Index documents
    indexer.index_directory(data_dir)

    # Interactive loop
    print("\n=== RAG System Ready ===")
    print("Enter your questions (or 'quit' to exit)")
    print("Examples:")
    print("- What is machine learning?")
    print("- Compare AI and deep learning")
    print("- What's your opinion on digital transformation?")
    print("- How do current technology trends relate to business strategy?")

    while True:
        try:
            query = input("\nQuery: ").strip()

            if query.lower() in ['quit', 'exit', 'q']:
                break

            if not query:
                continue

            print("\nProcessing...")
            start_time = time.time()

            response = rag_system.process_query(query)

            end_time = time.time()

            print(f"\n{'='*60}")
            print(f"Query Type: {response.query_type.value.upper()}")
            print(f"Strategy: {response.strategy_used}")
            print(f"Confidence: {response.confidence:.2f}")
            print(f"{'='*60}")
            print(f"\nAnswer:\n{response.answer}")

            print(f"\nSources ({len(response.sources)}):")
            for i, doc in enumerate(response.sources[:3], 1):
                source = doc.source.split('/')[-1] if '/' in doc.source else doc.source
                print(f"  {i}. {source}")

            if response.metadata and 'evaluation' in response.metadata:
                eval_data = response.metadata['evaluation']
                print(f"\nEvaluation Score: {eval_data.get('score', 'N/A')}/10")

            print(f"\nProcessing Time: {end_time - start_time:.2f} seconds")

        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            print(f"Error: {e}")



##Running the system

In [ ]:
if __name__ == "__main__":
    main()

=== Modular RAG System with ChromaDB and Llama3 ===

✓ Ollama connection successful
Setting up RAG system...
Initializing Llama3...
Initializing embeddings...
Setting up ChromaDB...
Indexing documents from ./my_documents...
Processing: my_documents/events.csv
  Added 152 chunks
Indexing complete!

=== RAG System Ready ===
Enter your questions (or 'quit' to exit)
Examples:
- What is machine learning?
- Compare AI and deep learning
- What's your opinion on digital transformation?
- How do current technology trends relate to business strategy?

Query: what is happening in september

Processing...

Query Type: FACTUAL
Strategy: factual
Confidence: 0.90

Answer:
According to the provided context, there are two events scheduled for September:

1. Craft sessions on the 4th of September from 6pm-9pm.
2. Another craft session on the 11th of September from 6pm-9pm.

These facts can be verified by reviewing the "Title" and "Date and Time" information provided in the context.

Sources (5):
  1. ev

# Evaluation

In [ ]:
import json
from pathlib import Path
from typing import List, Dict, Set
import pandas as pd

class TestRAGSystem:
    def __init__(self, rag_system, test_data_path: str):
        """
        Initialize the test class with the RAG system and test data path.

        Args:
            rag_system: Instance of the RAGSystem to test
            test_data_path: Path to the JSON file containing test data
        """
        self.rag_system = rag_system
        self.test_data_path = Path(test_data_path)
        self.test_data = self._load_test_data()

    def _load_test_data(self) -> List[Dict]:
        """Load test data from JSON file."""
        with open(self.test_data_path, "r") as f:
            return json.load(f)

    def _normalize_event(self, event: Dict) -> Set[int]:
        """Extract matching_csv_rows for comparison."""
        return set(event["matching_csv_rows"])

    def _extract_events_from_response(self, response: List) -> Set[int]:
        """
        Extract row numbers from the RAG system response's source documents.

        Args:
            response: List of Document objects from RAGSystem.retrieve_documents

        Returns:
            Set of row numbers from the metadata of retrieved documents
        """
        rows = set()
        for doc in response:
          # Extract row number from document metadata
          try:
              row = doc.metadata.get("row", None)
              if row is not None:
                  #print("row num", row, doc)
                  rows.add(int(row))
              else:
                #print("no metadata", doc)
                pass
          except (AttributeError, ValueError):
            continue


        return rows

    def _calculate_metrics(self, retrieved: Set[int], ground_truth: Set[int]) -> Dict:
        """
        Calculate precision, recall, and F1 score for retrieved row numbers.

        Args:
            retrieved: Set of retrieved row numbers
            ground_truth: Set of ground truth row numbers

        Returns:
            Dictionary with precision, recall, and F1 score
        """
        true_positives = len(retrieved.intersection(ground_truth))
        precision = true_positives / len(retrieved) if retrieved else 0.0
        recall = true_positives / len(ground_truth) if ground_truth else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        return {
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "true_positives": true_positives,
            "retrieved_count": len(retrieved),
            "ground_truth_count": len(ground_truth)
        }

    def test_retrieval(self) -> pd.DataFrame:
        """
        Run tests for all queries in the test data and store results in a DataFrame.
        Compares row numbers from retrieved documents with matching_csv_rows in test data.

        Returns:
            DataFrame containing test results for all queries
        """
        results = []
        for test_case in self.test_data:
            query = test_case["query"]
            ground_truth_rows = set()
            for event in test_case["ground_truth"]:
                ground_truth_rows.update(self._normalize_event(event))

            print(f"\nTesting query: {query}")
            response = self.rag_system.retrieve_documents(query)
            #print(f"Retrieved documents in test: {response}")
            retrieved_rows = self._extract_events_from_response(response)
            metrics = self._calculate_metrics(retrieved_rows, ground_truth_rows)

            print(f"Ground Truth Rows: {ground_truth_rows}")
            print(f"Retrieved Rows: {retrieved_rows}")
            print(f"Metrics: Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}, F1={metrics['f1_score']:.2f}")
            print(f"True Positives: {metrics['true_positives']}, Retrieved: {metrics['retrieved_count']}, Ground Truth: {metrics['ground_truth_count']}")

            results.append({
                "Query": query,
                "Precision": metrics["precision"],
                "Recall": metrics["recall"],
                "F1_Score": metrics["f1_score"],
                "True_Positives": metrics["true_positives"],
                "Retrieved_Count": metrics["retrieved_count"],
                "Ground_Truth_Count": metrics["ground_truth_count"],
                "Retrieved_Rows": "; ".join(map(str, sorted(retrieved_rows))),
                "Ground_Truth_Rows": "; ".join(map(str, sorted(ground_truth_rows)))
            })

        # Create DataFrame
        results_df = pd.DataFrame(results)
        print("\nTest Results Summary:")
        print(results_df[["Query", "Precision", "Recall", "F1_Score", "True_Positives", "Retrieved_Count", "Ground_Truth_Count"]])
        return results_df

def run_tests(rag_system, test_data_path: str) -> pd.DataFrame:
    """
    Run all tests for the RAG system and return results in a DataFrame.

    Args:
        rag_system: Instance of the RAGSystem to test
        test_data_path: Path to the JSON file containing test data

    Returns:
        DataFrame containing test results
    """
    tester = TestRAGSystem(rag_system, test_data_path)
    return tester.test_retrieval()

# Sep

In [ ]:
rag_system = RAGSystem(data_dir="./enterprise_data", persist_dir="./chroma_db",  debug=False, _k1=5, _k=10)
rag_system.index_documents()
results_df = run_tests(rag_system, "retrieval_test.json")
results_df.describe()

Indexing documents...
Successfully indexed 152 chunks from enterprise_data/events - Copy.csv

Testing query: What events are related to mental health awareness in May?
Ground Truth Rows: {1, 20, 5, 7, 9, 28}
Retrieved Rows: {1, 5, 9, 20, 28}
Metrics: Precision=1.00, Recall=0.83, F1=0.91
True Positives: 5, Retrieved: 5, Ground Truth: 6

Testing query: What events are happening on the 14th of May?
Ground Truth Rows: {16, 9, 10, 11, 12, 13, 14, 15}
Retrieved Rows: {2, 6, 72, 11, 46, 14, 15}
Metrics: Precision=0.43, Recall=0.38, F1=0.40
True Positives: 3, Retrieved: 7, Ground Truth: 8

Testing query: Is there a weekly meeting for Red Essex on the 19th May?
Ground Truth Rows: {45}
Retrieved Rows: {72, 2, 46}
Metrics: Precision=0.00, Recall=0.00, F1=0.00
True Positives: 0, Retrieved: 3, Ground Truth: 1

Testing query: When is the International Cafe for Mexican & Spanish Night?
Ground Truth Rows: {26}
Retrieved Rows: {26}
Metrics: Precision=1.00, Recall=1.00, F1=1.00
True Positives: 1, Retrie

,Precision,Recall,F1_Score,True_Positives,Retrieved_Count,Ground_Truth_Count
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.370855,0.299427,0.318837,1.111111,3.740741,4.037037
std,0.407904,0.332175,0.351752,1.310705,2.379279,2.457038
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
50%,0.250000,0.250000,0.200000,1.000000,3.000000,4.000000
75%,0.750000,0.414286,0.500000,1.000000,5.000000,6.000000
max,1.000000,1.000000,1.000000,5.000000,10.000000,8.000000


In [ ]:
if __name__ == "__main__":
    main()